In [5]:
import jax
import jax.numpy as jnp

from jax import grad

In [6]:
from jax import grad

def sum_logistic(x):
  return jnp.sum(1.0 / (1.0 + jnp.exp(-x)))

x_small = jnp.arange(3.)
derivative_fn = grad(sum_logistic)
print(x_small)
print(derivative_fn(x_small))

[0. 1. 2.]
[0.25       0.19661194 0.10499357]


In [15]:
from jax import random

#key = random.key(1701)
#key = random.rbg_key(0)#(1701)
key = random.PRNGKey(1701)

key1, key2 = random.split(key)
mat = random.normal(key1, (20, 10))
batched_x = random.normal(key2, (5, 10))

def apply_matrix(x):
  return jnp.dot(mat, x)

#print(batched_x)
for x in batched_x:
  print(apply_matrix(x))


[ 2.194634   -5.9014564  -0.80030835  3.0623093   4.8723454  -6.2820053
  2.2746496   5.109309   -6.387403    4.4732537   0.8514924   5.331101
 -2.8103292  -5.7421885   0.56605804  5.184789   -2.4608655  -1.5103221
 -0.8573533   4.39187   ]
[ 0.700902   -1.5472901  -5.3857756   0.71137357  0.25184524  3.9353704
  1.7903665   4.582093    0.2742404   5.8519993   3.992779    4.9032307
 -1.300314   -1.3217523   2.726924   -0.43353802 -0.7758179   3.1832604
  0.8794904   5.8944206 ]
[-1.5831186   2.7926424   5.2001114  -7.0306234  -0.5823697   0.8183627
  0.5250131  -4.4935637   3.4041622  -1.3305156  -0.3964243   1.2360106
 -2.496494    0.8443636   4.531971    3.54793    -0.27171394 -1.1906793
  5.9480853  -0.5166249 ]
[-0.40956816 -3.390621   -0.76107645 -2.594502   -0.9882018   0.14693908
  4.144276   -1.578219   -1.1291407  -0.5344701   0.3230201   3.1522849
 -2.6746552  -2.5621486   1.0059206   2.4140084  -0.41857228  0.81173867
  0.19573656  2.9980764 ]
[ 2.5354056  -1.4303145  -1.693

In [20]:
from jax import ShapedArray, jit

def f(x):
  if x < 3:
    return 3. * x ** 2
  else:
    return -4 * x

f = jit(f, static_argnums=(0,))

print(f(2.))
print(f(5.))



12.0
-20.0


In [25]:
def f(x, y):
    if x < 3:
        return y
    else:
        return y * 2


f_jitted = jit(f, static_argnums=(0,))

# 追踪时 y 的值会被固定
result = f_jitted(2, 5)
print(result)  

5


In [29]:
from jax.experimental import pjit


devices = jax.devices()

# 创建一个分片策略，将数组分割成两个部分，并分别放在两个设备上
sharding_spec = pjit.PartitionSpec(0)

@partial(pjit.pjit, in_axis_resources=sharding_spec, out_axis_resources=sharding_spec)
def parallel_function(x):
    return jnp.sin(x)

x = jnp.arange(5.0)

y = parallel_function(x)
print(y)


NameError: name 'partial' is not defined